<a href="https://colab.research.google.com/github/Fahin2000/Data-110/blob/main/project__2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
file_path = "/winemag-data-130k-v2-Copy1.csv"

# Project 1 : Finding wine preferences, Sommeliers with Tree based Machine learning models

In [ ]:
import subprocess

try:
    import kaggle
    print("Kaggle is installed.")
except ImportError:
    print("Kaggle is not installed. Installing now...")
    subprocess.check_call(["pip", "install", "kaggle"])

OSError: Could not find kaggle.json. Make sure it's located in /root/.config/kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/

In [ ]:
import os as os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi

if os.path.exists(file_path):
    print("file correctly installed", file_path)

# if kaggle.json exists
elif os.path.exists(os.path.expanduser("~/.kaggle/kaggle.json")):
    print("File does not exist, but now downloading from kaggle")
    api = KaggleApi()
    api.authenticate()
    api.dataset_download_files('zynicide/wine-reviews', path='.', unzip=True)
    if os.path.exists(file_path):
        print("File is here:", file_path)
    else:
        print("File in the wrong place pls check.")
else:
    print("please download the file / get kaggle API: https://www.kaggle.com/docs/api")

file correctly installed /winemag-data-130k-v2-Copy1.csv


In [ ]:
if os.path.exists(file_path):
    sommelier = pd.read_csv(file_path)
    print("somelier created")

else:
    print("youve got to fix the filepath")

somelier created


In [ ]:
 sommelier.head(3)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm


In [ ]:
sommelier.columns

In [ ]:
# Removing unnecessary columns
sommelier.drop(['Unnamed: 0', 'region_2',
       'taster_twitter_handle'], axis=1, inplace=True)

In [ ]:
# find vintage, year pattern is \b\d{4}\b
sommelier['vintage'] = sommelier['title'].str.extract(r'(\b\d{4}\b)')

In [ ]:
vintage_count = sommelier['vintage'].value_counts()
print(vintage_count)

In [ ]:
sommelier.isna().sum()

In [ ]:
# there are a handful of wines that lack country,
# province and one that lacks variety, there is limited value in keeping these so we will remove them
sommelier.dropna(subset=['variety','province','country','vintage'], inplace = True)
# checking to see if we were successful
sommelier.isna().sum()

In [ ]:
# Checking types
print(sommelier.dtypes)

In [ ]:
# Making year an integer
sommelier['vintage'] = sommelier['vintage'].astype(int)


In [ ]:
# Making removing obvious mistakes
# Wines have been discovered from as early as the 15th century I believe, but they are only curiosities and have little to no acutal flavour value, they are therefore not included in our search
sommelier = sommelier[(sommelier['vintage'] >= 1600) & (sommelier['vintage'] <= 2024)]

In [ ]:
sns.relplot(x = 'points',
            y = "price",
            data = sommelier,
            kind= "scatter",
            hue = 'country',
            )
#plt.title
#plt.xlabel
#plt.ylabel
plt.show()

In [ ]:
# It is interesting that the wine reviewers clearly do not review wine that is below "80%", In reality they operate on a 1 to 20 scale, and then add 80 points
sns.relplot(x = 'points',
            y = "price",
            data = sommelier,
            kind= "scatter",
            hue = 'vintage',
            )
#plt.title
#plt.xlabel
#plt.ylabel
plt.show()

In [ ]:
print( sommelier['taster_name'].unique())

In [ ]:
# this code colours all the points related to a specific sommelier in blue
colors = sommelier['taster_name'].apply(lambda x: 'blue'
                                        if x == 'Kerin O’Keefe'
                                        else 'gray')

In [ ]:
# 3D plot of the taster named in the above chunk and how they map onto price, score and vintage age
from mpl_toolkits.mplot3d import Axes3D

sns.set_style("whitegrid", {'axes.grid' : False})

fig = plt.figure(figsize=(6,6))

# ax = Axes3D(fig) Did not work

ax = fig.add_subplot(111, projection='3d')

c = 'price'
a = 'points'
b = 'vintage'

x = sommelier[a]
y = sommelier[b]
z = sommelier[c]


ax.scatter(x, y, z, c=colors, marker='o')


ax.set_xlabel(a)
ax.set_ylabel(b)
ax.set_zlabel(c)


plt.show()

There is clustering here, and trying various tasters resulted in different clusters. For example 'Kerin O’Keefe' appears to rate wines from the early 20th century between 85 and 90 points but rarely higher. His more recent post 2000 wines have a much greater spread.

In [ ]:
print(sommelier['winery'].unique())
print(sommelier['province'].unique())
print(sommelier['variety'].unique())
print(sommelier['country'].unique())

In [ ]:
sommelier.nunique()

## Machine Learning component





In [ ]:
# Getting dummies and getting rid of unwanted features

# So this will be a controversial choice, less so than age, specific years are often associated with good harvest and therefore makes vintage year a categorical variable rather than a clearly numeric one
sommelier['wine_age'] = 2024 - sommelier['vintage']
sommelier_ml = sommelier.copy()
# We think that province, winery, and variety (especially variety) would probably be a good feature, but unfortunately it might make the whole thing a little too complex.
sommelier_ml.drop(['designation', 'description', 'title','region_1','province','winery','vintage','variety'], axis=1, inplace=True)

sommelier_ml = pd.get_dummies(sommelier_ml, columns=['country'], drop_first=True)

print(sommelier_ml.head(3))

In [ ]:
sommelier_ml.isna().sum()

In [ ]:
sommelier_ml['reviewed_by_kerin'] = (sommelier_ml['taster_name'] == 'Kerin O’Keefe').astype(int)

In [ ]:
sommelier_ml['price'].fillna(sommelier_ml['price'].median(), inplace=True)
sommelier_ml['taster_name'].fillna(False, inplace=True)

In [ ]:
sommelier_ml.drop('taster_name', axis=1, inplace=True)

In [ ]:
sommelier_ml.isna().sum()

In [ ]:
from sklearn.model_selection import train_test_split

X = sommelier_ml.drop(['reviewed_by_kerin'], axis=1)
y = sommelier_ml['reviewed_by_kerin']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

# Instantiate gb
gb = GradientBoostingRegressor(max_depth=10,
            n_estimators=200,
            random_state=2)
print('complete')

In [ ]:
# Import mean_squared_error as MSE
from sklearn.metrics import mean_squared_error as MSE

# Compute MSE
mse_test = MSE(y_test, y_pred)

# Compute RMSE
rmse_test = mse_test**(1/2)

# Print RMSE
print('complete')
print('Test set RMSE of gb: {:.3f}'.format(rmse_test))

## Trying SGB

In [ ]:
# Import GradientBoostingRegressor for second try
from sklearn.ensemble import GradientBoostingRegressor

# Instantiate sgbr
sgbr = GradientBoostingRegressor(max_depth= 10,
            subsample= 0.9,
            max_features= 0.75,
            n_estimators=200,
            random_state=2)

print('done')

In [ ]:
# Fit sgbr to the training set
sgbr.fit(X_train,y_train)

# Predict test set labels
y_pred = sgbr.predict(X_test)
print('done')

In [ ]:
# Import mean_squared_error as MSE
from sklearn.metrics import mean_squared_error as MSE

# Compute test set MSE
mse_test = MSE(y_test, y_pred)
# Compute test set RMSE
rmse_test = mse_test**(1/2)


# Print rmse_test
print('done')
print('Test set RMSE of sgbr: {:.3f}'.format(rmse_test))

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE
from sklearn.model_selection import cross_val_score

# Set SEED for reproducibility
SEED = 123

X = sommelier_ml.drop(['reviewed_by_kerin'], axis=1)
y = sommelier_ml['reviewed_by_kerin']


# Split the data into 70% train and 30% test
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=SEED)


print('done')

In [ ]:
# Instantiate a DecisionTreeRegressor dt
dt = DecisionTreeRegressor(max_depth=10,
                           min_samples_leaf=0.14,
                           random_state=SEED)

MSE_CV = - cross_val_score(dt,
                          X_train,
                          y_train,
                          cv = 10,
                          scoring = 'neg_mean_squared_error',
                          n_jobs = -1)
dt.fit(X_train, y_train)
y_predict_train = dt.predict(X_train)
y_predict_test = dt.predict(X_test)
print('done')

In [ ]:
print('CV MSE: {:.2f}'.format(MSE_CV.mean()))

In [ ]:
print('Train MSE: {:.2f}'.format(MSE(y_train, y_predict_train)))

In [ ]:
print('Test MSE: {:.2f}'.format(MSE(y_test, y_predict_test)))

In [ ]:
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_predict_test)
print(f"R^2: {r2:.2f}")